In [1]:
import numpy as np
import pandas as pd

In [50]:
train_df = pd.read_csv("train.csv")

train_df

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False
...,...,...,...,...
135304,135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,False
135305,135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....,False
135306,135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati...",False
135307,135307,wowcream.ru,Купить It's Skin Сыворотка питательная Power 1...,False


In [3]:
len(train_df)

135309

In [4]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [5]:
len(test_df)

165378

In [6]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ilya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import re
import pymorphy2

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.preprocessing import StandardScaler

In [9]:
TOKEN_PATTERN = "[а-яё]+"
SEED = 42

In [10]:
stop_words = nltk.corpus.stopwords.words("russian")

In [11]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values

In [12]:
def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())

In [13]:
lemmatizer = pymorphy2.MorphAnalyzer()

lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

In [14]:
X_train_tokenized = [tokenize(text) for text in X_train]

X_train_lemmatized = [[lemmatize(token) for token in text] for text in X_train_tokenized]

X_train_cleared = [" ".join([token for token in text if token not in stop_words]) for text in X_train_lemmatized]

In [116]:
X_train_cleared

['экс министр экономика молдова глава мидэи цель который сделать республика проситель донор избегать долгий нахождение н',
 'песня стать известный многий телезритель благодаря сериал диверсант',
 'банши сезон серия бремя красота смотреть онлайн',
 'бесить картинка',
 'новомосковск сыграть следж хоккеист алексинский звезда сборная китай',
 'салат корейский морковь копчёный курица кукуруза сухарик калорийность белка жир углевод',
 'угловой пневмогайковерт екатеринбург купить цена отзыв характеристика фото инструкция',
 'табачный магазин мир табак курительный трубка курительный труба',
 '',
 'р одесса ростов дон сентябрь фаворит чп',
 'огэ математик функция задача ответ готовый',
 'мировой экономика схема таблица млн результат поиск',
 'кримпай мать сын страница смотреть порно мультик порно комикс хентай онлайн бесплатно',
 'мария шорох право выбор академия си читать онлайн',
 'днепр наслать пропасть летний девушка',
 'мир',
 'слава медяник стучать колесо',
 'инженерный институт казанский

In [117]:
X_test_tokenized = [tokenize(text) for text in X_test]

X_test_lemmatized = [[lemmatize(token) for token in text] for text in X_test_tokenized]

X_test_cleared = [" ".join([token for token in text if token not in stop_words]) for text in X_test_lemmatized]

In [118]:
X_test_cleared

['шестой кассационный суд самара начать работа разный здание фото коммерсантъ',
 'индексация алименты случай производиться каков порядок правило процедура',
 'женщина империя мех',
 'небритый волосатый киска порно весь страна национальность онлайн',
 '',
 'вакансия мерчендайзер визитный рязань группа компания поиск работа городработ ру',
 'смартфон чёрный купить интернет магазин фотосклад ру цена отзыв видео обзор',
 'духов шкаф цена москва',
 'вакансия аналитик департамент внедрение информационный система платформа',
 'торрент',
 'убрать запах выгребной ям',
 'озаботить старый баба порно фотосессия',
 'смотреть онлайн н',
 'ещё видеть айдаприкол',
 'развивать игрушка весы тюлень солнышко игрушка ребёнок год',
 'наружный блок',
 'дочь кадыров признать самый одарённый ученик чечня яплакалъ',
 '',
 'рекомендация учёт жёсткость диафрагма стальной профилировать настил покрытие',
 'продажа год уссурийск отправка регион подробность телефон вап сап акпп л вд дизель дилер',
 'ерлан тажибаев бе

# CountVectoriser

In [17]:
vectorizer = CountVectorizer(lowercase=True, stop_words=stop_words)

X_train_vectorized = vectorizer.fit_transform(X_train)

X_train_vectorized

<135309x189711 sparse matrix of type '<class 'numpy.int64'>'
	with 1120352 stored elements in Compressed Sparse Row format>

## LogisticRegression

In [18]:
model = LogisticRegression()

In [19]:
%time

model.fit(X_train_vectorized, y_train)

y_pred = model.predict(X_train_vectorized)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [20]:
f1_score(y_train, y_pred)

0.9746839317966144

In [21]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["target"] = model.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("log_reg.csv", index=False)

!cat ml_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: write error: Broken pipe


## SGDClassifier

In [22]:
model = SGDClassifier()

In [23]:
%time

model.fit(X_train_vectorized, y_train)

y_pred = model.predict(X_train_vectorized)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 5.72 µs


In [24]:
f1_score(y_train, y_pred)

0.957971733201091

In [25]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["target"] = model.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("SGD.csv", index=False)

!cat ml_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: write error: Broken pipe


## ComplementNB

In [26]:
model = ComplementNB()

In [27]:
%time

model.fit(X_train_vectorized, y_train)

y_pred = model.predict(X_train_vectorized)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 5.25 µs


In [28]:
f1_score(y_train, y_pred)

0.8324219348903767

## MultinomialNB

In [29]:
model = MultinomialNB()

In [30]:
%time

model.fit(X_train_vectorized, y_train)

y_pred = model.predict(X_train_vectorized)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.38 µs


In [31]:
f1_score(y_train, y_pred)

0.9097784676167632

# TfidfVectorizer

In [ ]:
# tfidf_vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(2, 4), analyzer='char', min_df=5, max_df=0.8)

# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.fit_transform(X_test)

In [68]:
def parse_url(url):
    return url[:url.rfind('.')].replace('www.', '')

In [120]:
X_train_url = (train_df["url"].apply(lambda x: parse_url(x)) + " " + train_df["title"]).values
X_test_url = (test_df["url"].apply(lambda x: parse_url(x)) + " " + test_df["title"]).values

X_train_url_cleared = train_df["url"].apply(lambda x: parse_url(x)).values + " " + X_train_cleared
X_test_url_cleared = test_df["url"].apply(lambda x: parse_url(x)).values + " " + X_test_cleared

In [115]:
X_train_url_cleared

array(['m.kpэкс министр экономика молдова глава мидэи цель который сделать республика проситель донор избегать долгий нахождение н',
       'kpпесня стать известный многий телезритель благодаря сериал диверсант',
       'fanserialsбанши сезон серия бремя красота смотреть онлайн', ...,
       'topclassiccarsforsale',
       'wowcreamкупить сыворотка питательный мл москва',
       'ubuтехнический спирт канистра флакон купить москва'], dtype=object)

In [121]:
X_test_url_cleared

array(['kommersant шестой кассационный суд самара начать работа разный здание фото коммерсантъ',
       'urexpert индексация алименты случай производиться каков порядок правило процедура',
       'imperimeha женщина империя мех', ...,
       'xn----8sbnqchpeeeth администрация лесной район тверской область годовщина снятие блокада',
       'www-sunhome-ru.cdn.ampproject сонник изменение сознание сниться изменение сознание видеть сон сонник дом солнце',
       'virtual-hockey строительство база команда гранд рапидс гриффинс гранд рапидс сша'],
      dtype=object)

## LogisticRegression

In [122]:
char_tfidf_log_reg = Pipeline([
    (
        'vectorizer',
        TfidfVectorizer(
            lowercase=True, ngram_range=(2, 4), analyzer='char',
            min_df=5, max_df=0.8
        )
    ),
    ('clf', LogisticRegression(class_weight='balanced', random_state=SEED))
])

In [124]:
%time

char_tfidf_log_reg.fit(X_train_url_cleared, y_train)

y_pred = char_tfidf_log_reg.predict(X_train_url_cleared)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 3.81 µs


/home/ilya/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [125]:
f1_score(y_train, y_pred)

0.9602154273801251

In [107]:
test_df["target"] = char_tfidf_log_reg.predict(X_test_url).astype(bool)

test_df[["id", "target"]].to_csv("log_reg_tfidf.csv", index=False)

!cat ml_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: write error: Broken pipe


## SGDClassifier

In [126]:
char_tfidf_sgd = Pipeline([
    (
        'vectorizer',
        TfidfVectorizer(
            lowercase=True, ngram_range=(2, 4), analyzer='char',
            min_df=5, max_df=0.8
        )
    ),
    ('clf', SGDClassifier(random_state=SEED, loss='log', class_weight='balanced'))
])

In [129]:
%time

char_tfidf_sgd.fit(X_train_url, y_train)

y_pred = char_tfidf_sgd.predict(X_train_url)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs


In [130]:
f1_score(y_train, y_pred)

0.9602877355686344

In [111]:
test_df["target"] = char_tfidf_sgd.predict(X_test_url).astype(bool)

test_df[["id", "target"]].to_csv("SGD_tfidf.csv", index=False)

!cat ml_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: write error: Broken pipe
